<a href="https://colab.research.google.com/github/proteinovayas13/My_works/blob/main/%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D0%B2_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа в PySpark

1. Запустила кластер локально.

2. Выполнила базовые методы map, filter, join, fold, sortByKey, union на собственных примерах.

In [ ]:
!pip install pyspark

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,510 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backp

In [ ]:
!pip install findspark==1.4.2 pyspark==3.4.1

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark import RDD
from pyspark.sql import SparkSession

In [ ]:
#Запускаю локальный кластер PySpark

spark = SparkSession.builder \
    .appName("BasicSparkMethods") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=BasicSparkMethods>

In [ ]:
#  Датафрейм со студентами
students_data = [
    (1, "Анна", "Иванова", "ФИИТ", 21, 4.5),
    (2, "Петр", "Сидоров", "ПМИ", 22, 4.2),
    (3, "Мария", "Петрова", "ФИИТ", 20, 4.8),
    (4, "Алексей", "Кузнецов", "ИВТ", 23, 3.9),
    (5, "Елена", "Смирнова", "ПМИ", 21, 4.1),
    (6, "Дмитрий", "Васильев", "ФИИТ", 22, 4.6)
]

students_schema = ["id", "first_name", "last_name", "faculty", "age", "gpa"]
students_df = spark.createDataFrame(students_data, students_schema)
students = students_df.rdd

#  Датафрейм с курсами
courses_data = [
    (101, "Математический анализ", "ФИИТ", 5),
    (102, "Программирование на Python", "ИВТ", 4),
    (103, "Базы данных", "ФИИТ", 4),
    (104, "Алгоритмы и структуры данных", "ПМИ", 5),
    (105, "Веб-разработка", "ИВТ", 3),
    (106, "Машинное обучение", "ФИИТ", 5)
]

courses_schema = ["course_id", "course_name", "faculty", "credits"]
courses_df = spark.createDataFrame(courses_data, courses_schema)
courses = courses_df.rdd

# Датафрейм с оценками
grades_data = [
    (1, 101, 5), (1, 103, 4), (1, 106, 5),
    (2, 104, 4), (2, 101, 3),
    (3, 101, 5), (3, 103, 5), (3, 106, 4),
    (4, 102, 4), (4, 105, 5),
    (5, 104, 4), (5, 101, 4),
    (6, 101, 5), (6, 103, 5), (6, 106, 5)
]

grades_schema = ["student_id", "course_id", "grade"]
grades_df = spark.createDataFrame(grades_data, grades_schema)
grades = grades_df.rdd


# Map

In [ ]:
studentsReversed = students.map(lambda students: students[::-1])
studentsReversed

PythonRDD[57] at RDD at PythonRDD.scala:53

In [ ]:
studentsReversed.collect()

[(4.5, 21, 'ФИИТ', 'Иванова', 'Анна', 1),
 (4.2, 22, 'ПМИ', 'Сидоров', 'Петр', 2),
 (4.8, 20, 'ФИИТ', 'Петрова', 'Мария', 3),
 (3.9, 23, 'ИВТ', 'Кузнецов', 'Алексей', 4),
 (4.1, 21, 'ПМИ', 'Смирнова', 'Елена', 5),
 (4.6, 22, 'ФИИТ', 'Васильев', 'Дмитрий', 6)]

In [ ]:
coursesReversed = courses.map(lambda courses: courses[::-1])
coursesReversed.collect()

[(5, 'ФИИТ', 'Математический анализ', 101),
 (4, 'ИВТ', 'Программирование на Python', 102),
 (4, 'ФИИТ', 'Базы данных', 103),
 (5, 'ПМИ', 'Алгоритмы и структуры данных', 104),
 (3, 'ИВТ', 'Веб-разработка', 105),
 (5, 'ФИИТ', 'Машинное обучение', 106)]

In [ ]:
gradesReversed = grades.map(lambda grades: grades[::-1])
gradesReversed.collect()

[(5, 101, 1),
 (4, 103, 1),
 (5, 106, 1),
 (4, 104, 2),
 (3, 101, 2),
 (5, 101, 3),
 (5, 103, 3),
 (4, 106, 3),
 (4, 102, 4),
 (5, 105, 4),
 (4, 104, 5),
 (4, 101, 5),
 (5, 101, 6),
 (5, 103, 6),
 (5, 106, 6)]

In [ ]:
student_ids = students.map(lambda x: x[0])  # student
grade_student_ids = grades.map(lambda x: x[0])  # student_id из оценок

In [ ]:
student_ids.collect()

[1, 2, 3, 4, 5, 6]

In [ ]:
grade_student_ids.collect()

[1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 6]

# filter

In [ ]:
#Студенты с GPA больше 4.0
good_students = students.filter(lambda x: x[5] > 4.0)

for s in good_students.collect():
    print(f"  {s[1]} {s[2]} - GPA: {s[5]}")

  Анна Иванова - GPA: 4.5
  Петр Сидоров - GPA: 4.2
  Мария Петрова - GPA: 4.8
  Елена Смирнова - GPA: 4.1
  Дмитрий Васильев - GPA: 4.6


In [ ]:
# Студенты младше 22 лет
young_students = students.filter(lambda x: x[4] < 22)

for s in young_students.collect():
    print(f"  {s[1]} {s[2]} - возраст: {s[4]}")

  Анна Иванова - возраст: 21
  Мария Петрова - возраст: 20
  Елена Смирнова - возраст: 21


In [ ]:
# Студенты факультета ФИИТ
fiit_students = students.filter(lambda x: x[3] == "ФИИТ")

for s in fiit_students.collect():
    print(f"  {s[1]} {s[2]}")

  Анна Иванова
  Мария Петрова
  Дмитрий Васильев


In [ ]:
# Студенты ФИИТ с GPA выше 4.0
good_fiit = students.filter(lambda x: x[3] == "ФИИТ" and x[5] > 4.0)

for s in good_fiit.collect():
    print(f"  {s[1]} {s[2]} - GPA: {s[5]}")

  Анна Иванова - GPA: 4.5
  Мария Петрова - GPA: 4.8
  Дмитрий Васильев - GPA: 4.6


# JOIN

In [ ]:
# Преобразуем ID в (id, 1)
student_id_key = student_ids.map(lambda x: (x, 1))
grade_id_key = grade_student_ids.map(lambda x: (x, 1))

# Выполняем inner join - получим только общие элементы
common_students_join = student_id_key.join(grade_id_key)
common_students_join.collect()

[(4, (1, 1)),
 (4, (1, 1)),
 (1, (1, 1)),
 (1, (1, 1)),
 (1, (1, 1)),
 (5, (1, 1)),
 (5, (1, 1)),
 (2, (1, 1)),
 (2, (1, 1)),
 (6, (1, 1)),
 (6, (1, 1)),
 (6, (1, 1)),
 (3, (1, 1)),
 (3, (1, 1)),
 (3, (1, 1))]

# fold

In [ ]:
# Найти самый длинный курс
longest_name = courses.fold("",
                                lambda longest, name: name if len(name) > len(longest) else longest)
print(f"Самый длинный курс: {longest_name}")

Самый длинный курс: Row(course_id=101, course_name='Математический анализ', faculty='ФИИТ', credits=5)


In [ ]:
# Конкатенация всех имен студентов
names = students.map(lambda x: x[1])  # берем только имена
all_names = names.fold("", lambda text, name: text + ", " + name if text else name)
print(f"Все имена через запятую: {all_names}")

Все имена через запятую: Анна, Петр, Мария, Алексей, Елена, Дмитрий


# sortByKey

In [ ]:
# Сортировка оценок по курсу
grades_by_course = grades.map(lambda x: (x[1], (x[0], x[2])))  # (course_id, (student_id, grade))
sorted_by_course = grades_by_course.sortByKey()
print(" Оценки сгруппированные по курсам:")
for course_id, (student_id, grade) in sorted_by_course.collect():
    print(f"   Курс {course_id}: студент {student_id}, оценка {grade}")

 Оценки сгруппированные по курсам:
   Курс 101: студент 1, оценка 5
   Курс 101: студент 2, оценка 3
   Курс 101: студент 3, оценка 5
   Курс 101: студент 5, оценка 4
   Курс 101: студент 6, оценка 5
   Курс 102: студент 4, оценка 4
   Курс 103: студент 1, оценка 4
   Курс 103: студент 3, оценка 5
   Курс 103: студент 6, оценка 5
   Курс 104: студент 2, оценка 4
   Курс 104: студент 5, оценка 4
   Курс 105: студент 4, оценка 5
   Курс 106: студент 1, оценка 5
   Курс 106: студент 3, оценка 4
   Курс 106: студент 6, оценка 5


In [ ]:
# Сортировка оценок по студенту
grades_by_student = grades.map(lambda x: (x[0], (x[1], x[2])))  # (student_id, (course_id, grade))
sorted_grades = grades_by_student.sortByKey()
print("\n5. Оценки сгруппированные по студентам:")
for student_id, (course_id, grade) in sorted_grades.collect():
    print(f"   Студент {student_id}: курс {course_id}, оценка {grade}")


5. Оценки сгруппированные по студентам:
   Студент 1: курс 101, оценка 5
   Студент 1: курс 103, оценка 4
   Студент 1: курс 106, оценка 5
   Студент 2: курс 104, оценка 4
   Студент 2: курс 101, оценка 3
   Студент 3: курс 101, оценка 5
   Студент 3: курс 103, оценка 5
   Студент 3: курс 106, оценка 4
   Студент 4: курс 102, оценка 4
   Студент 4: курс 105, оценка 5
   Студент 5: курс 104, оценка 4
   Студент 5: курс 101, оценка 4
   Студент 6: курс 101, оценка 5
   Студент 6: курс 103, оценка 5
   Студент 6: курс 106, оценка 5


In [ ]:
# Сортировка студентов по возрасту
students_by_age = students.map(lambda x: (x[4], (x[1], x[2])))  # (возраст, (имя, фамилия))
sorted_by_age = students_by_age.sortByKey()
print("Студенты по возрасту:")
for age, (name, lastname) in sorted_by_age.collect():
    print(f"   Возраст {age}: {name} {lastname}")

Студенты по возрасту:
   Возраст 20: Мария Петрова
   Возраст 21: Анна Иванова
   Возраст 21: Елена Смирнова
   Возраст 22: Петр Сидоров
   Возраст 22: Дмитрий Васильев
   Возраст 23: Алексей Кузнецов


# union

In [ ]:
all_tables = students.union(courses).union(grades)
print(f"Всего записей в объединенной таблице: {all_tables.count()}")

for row in all_tables.take(5):
    print(f"   {row}")

Всего записей в объединенной таблице: 27
   Row(id=1, first_name='Анна', last_name='Иванова', faculty='ФИИТ', age=21, gpa=4.5)
   Row(id=2, first_name='Петр', last_name='Сидоров', faculty='ПМИ', age=22, gpa=4.2)
   Row(id=3, first_name='Мария', last_name='Петрова', faculty='ФИИТ', age=20, gpa=4.8)
   Row(id=4, first_name='Алексей', last_name='Кузнецов', faculty='ИВТ', age=23, gpa=3.9)
   Row(id=5, first_name='Елена', last_name='Смирнова', faculty='ПМИ', age=21, gpa=4.1)
